In [0]:
%run "./nyc_taxi_conn"

##TRIP-TYPE DATA

In [0]:
df_trip_type = spark.read.format('csv') \
    .option('header', True) \
        .option('inferSchema', True) \
            .load(f'{bronze}/trip-type')

# display(df_trip_type)

In [0]:
df_trip_type = df_trip_type.withColumnRenamed('description', 'trip-des')
# df_trip_type.display()

In [0]:
df_trip_type.write.format('parquet') \
    .option('path', f'{silver}/trip-type') \
        .option('mode', 'overwrite') \
            .save()

##TRIP-ZONE DATA

In [0]:
df_trip_zone = spark.read.format('csv') \
    .option('header', True) \
        .option('inferSchema', True) \
            .load(f'{bronze}/trip-zone')

# df_trip_zone.display()

In [0]:
df_trip_zone = df_trip_zone.withColumn('Zone1', split(col('Zone'), '/')[0]) \
    .withColumn('Zone2', split(col('Zone'), '/')[1])

# df_trip_zone.display()

In [0]:
df_trip_zone.write.format('parquet') \
    .option('path', f'{silver}/trip-zone') \
        .option('mode', 'overwrite') \
            .save()

##TRIP DATA (2024)

In [0]:
myschema = '''
	VendorID BIGINT,
	lpep_pickup_datetime TIMESTAMP,
	lpep_dropoff_datetime TIMESTAMP,
	store_and_fwd_flag STRING,
	RatecodeID BIGINT,
	PULocationID BIGINT,
	DOLocationID BIGINT,
	passenger_count BIGINT,
	trip_distance DOUBLE,
	fare_amount DOUBLE,
	extra DOUBLE,
	mta_tax DOUBLE,
	trip_amount DOUBLE,
	tolls_amount DOUBLE,
	ehail_fee DOUBLE,
	improvement_surcharge DOUBLE,
	total_amount DOUBLE,
	payment_type BIGINT,
	trip_type BIGINT,
	congestion_surcharge DOUBLE
'''

In [0]:
df_trip_data = spark.read.format('parquet') \
    .option('recursiveFileLookup', True) \
        .option('header', True) \
            .schema(myschema) \
                .load(f'{bronze}/trip-data')
# df_trip_data.display()

In [0]:
df_trip_data = df_trip_data.withColumn('to_date', to_date(col('lpep_pickup_datetime'))) \
    .withColumn('year', year(col('lpep_pickup_datetime'))) \
        .withColumn('month', month(col('lpep_pickup_datetime')))
    
# df_trip_data.display()

In [0]:
df_trip_data.write.format('parquet') \
    .option('path', f'{silver}/trip-data') \
        .option('mode', 'overwrite') \
            .save()